In [1]:
class IPv4:
    def __init__(self, part1:int, part2:int, part3:int, part4:int, mask: int | str) -> None:
        self.part1:int = part1
        self.part2:int = part2
        self.part3:int = part3
        self.part4:int = part4
        if type(mask) == int:
            self.cidr:int = mask
            self.mask:str = self.cidrToMask(mask)
        elif type(mask) == str:
            self.mask:str = mask
            self.cidr:int = self.maskToCIDR()
        else:
            self.cidr:int = 0
            self.mask:str = "255.255.255.255"
        self.rangeSize: int = 2**(32-self.cidr)
        self.usableRangeSize: int = self.rangeSize - 2
        range_addresses = self.subnetRangeAdresses()
        self.first_sb_adress: tuple[int] = range_addresses[0]
        self.last_sb_adress: tuple[int] = range_addresses[1]

    def __str__(self) -> str:
        return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}"

    def __format__(self, format_spec: str) -> str:
        match format_spec:
            case "cidr":
                return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}/{self.cidr}"
            case "mask":
                return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}:{self.mask}"
        return self.__str__()
    
    def cidrToMask(self, cidr: int) -> str:
        mask = 0
        for i in range(cidr):
            mask |= (1<<(31-i))
        return(f"{mask>>24}.{mask>>16&0xFF}.{mask>>8&0xFF}.{mask&0xFF}")

    def subnetRangeAdresses(self) -> tuple[int]:
        first_sb_adress = (self.part1, self.part2, self.part3, self.part4)
        last_sb_adress = (self.part1, self.part2, self.part3, self.part4)
        mask = self.mask
        mask0, mask8, mask16, mask24 = map(int, mask.split("."))
        size = 0
        offset = 0
        if 32 > self.cidr >= 24:
            size = 2**(32 - self.cidr)
            offset = self.part4 % size
            first_sb_adress = (self.part1, self.part2, self.part3, self.part4 - offset)
            last_sb_adress = (self.part1, self.part2, self.part3, self.part4 - offset + size - 1)
        elif 24 > self.cidr >= 16:
            size = 2**(24 - self.cidr)
            offset = self.part3 % size
            first_sb_adress = (self.part1, self.part2, self.part3 - offset, 0)
            last_sb_adress = (self.part1, self.part2, self.part3 - offset + size - 1, 255)
        elif 16 > self.cidr >= 8:
            size = 2**(16 - self.cidr)
            offset = self.part2 % size
            first_sb_adress = (self.part1, self.part2 - offset, 0, 0)
            last_sb_adress = (self.part1, self.part2 - offset + size - 1, 255, 255)
        elif 8 > self.cidr >= 0:
            size = 2**(8 - self.cidr)
            offset = self.part1 % size
            first_sb_adress = (self.part1 - offset, 0, 0, 0)
            last_sb_adress = (self.part1 - offset + size - 1, 255, 255, 255)
        return (first_sb_adress, last_sb_adress)

    def networkAddress(self) -> tuple[int]:
        mask_parts = list(map(int, self.mask.split('.')))
        network_part1 = self.part1 & mask_parts[0]
        network_part2 = self.part2 & mask_parts[1]
        network_part3 = self.part3 & mask_parts[2]
        network_part4 = self.part4 & mask_parts[3]
        return (network_part1, network_part2, network_part3, network_part4)

    def maskToCIDR(self) -> int:
        mask_parts = self.mask.split('.')
        cidr_count = 0
        for part in mask_parts:
            binary_representation = format(int(part), '08b')
            cidr_count += binary_representation.count('1')
        return cidr_count


In [2]:
ip1:IPv4 = IPv4(91,59,83,39,18)
print(ip1.cidr)
print(ip1.mask)
print(ip1.rangeSize)
print(ip1.usableRangeSize)
print(ip1.cidrToMask(18))


18
255.255.192.0
16384
16382
255.255.192.0


In [3]:
print(ip1.first_sb_adress,ip1.last_sb_adress,sep='->')

(91, 59, 64, 0)->(91, 59, 127, 255)


In [4]:
ip23:IPv4 = IPv4(0,0,0,0,"255.255.255.128")
print(ip23.cidr)

25
